In [5]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

  Activating project at `/data/davide/Servizi cloud/Dropbox/Ferracin_PhD/TDVP_VecRho_Davide`


In [6]:
using DelimitedFiles
using Permutations
using TimeEvoVecMPS
using VectorizedRho
using ITensors
include("TDVP_lib_VecRho.jl")

stretchBondDim (generic function with 1 method)

In [7]:
Nenv = 20;
sys = siteind("HvS=1/2");
env = siteinds("HvOsc",Nenv;dim=6);
sysenv = vcat(sys,env);
initSys = "Up";
initEnv = "0";
initSysEnv = vcat([initSys],[initEnv for i in 1:Nenv]);
psi0 = productMPS(sysenv,initSysEnv);
println("OK")
#...and we have the initial state....

OK


Extend the bond dimension (we are going to use TDVP1) and check overlap with the original factorized state.

In [8]:
psi_ext,overlap = stretchBondDim(psi0,12);
overlap

1.0000000000000007 + 0.0im

An example of how measurements should be used when dealing with vectorized density matrices.

In [24]:
V = ITensor(1.)
for j in 1:21
    print(j)
    #V *= psi_ext[j]* (j!=2 ? state("vecId",sysenv[j]) : state("vecId",sysenv[j]))
    V *= psi_ext[j] * state("vecId",sysenv[j])
end
scalar(V)


123456789101112131415161718192021

1.0000000000000042 + 0.0im

The function for the definition of the MPO associated to the full Lindblad evolution has been included in the TDVP_lib_VecRho.jl file

In [10]:
#createMPO(sysenv, eps::Float64, delta::Float64, freqfile::String, coupfile::String,
#MC_alphafile::String,MC_betafile::String,MC_coupfile::String,omega::Float64; kwargs...)
appo = createMPOVecRho(sysenv,69.,0.,
"../TDVP/Chain_mappings/WSCP_MC_T0_freqs.dat",
"../TDVP/Chain_mappings/WSCP_MC_T0_coups.dat",
"../TDVP/MC_Pars/alphas_6.dat",
"../TDVP/MC_Pars/betas_6.dat",
"../TDVP/MC_Pars/coupls_6.dat",500.);

pmtx = (1)(2)(3)(4)(5)(6)
80.70813909201401

Just keep an eye on the MPO bond dimension. As is, it seems that there are larger bond dimensions in between the oscillators in the closure

In [11]:
[dim(linkind(appo,j)) for j in 1:length(appo)-1];
[dim(linkind(psi_ext,j)) for j in 1:length(psi_ext)-1];


In [12]:
vobs = []
push!(vobs, opPos("Norm",1))
push!(vobs,opPos("vecσx",1))
push!(vobs,opPos("vecσz",1))
nchain = collect(1:1:20)
for i in nchain
   push!(vobs,opPos("vecN",i+1))
end

In [13]:
psi=deepcopy(psi_ext)
timestep = 0.0001
tmax = 0.02
cbT = LocalMeasurementCallbackTama(vobs,sysenv,timestep)

LocalMeasurementCallback
Operators: opPos[opPos("Norm", 1), opPos("vecσx", 1), opPos("vecσz", 1), opPos("vecN", 2), opPos("vecN", 3), opPos("vecN", 4), opPos("vecN", 5), opPos("vecN", 6), opPos("vecN", 7), opPos("vecN", 8)  …  opPos("vecN", 12), opPos("vecN", 13), opPos("vecN", 14), opPos("vecN", 15), opPos("vecN", 16), opPos("vecN", 17), opPos("vecN", 18), opPos("vecN", 19), opPos("vecN", 20), opPos("vecN", 21)]
No measurements performed


In [14]:
tdvp1!(psi,appo,timestep,tmax, hermitian = false, normalize = false, callback=cbT, progress=false, krylovdim = 10, exp_tol = 10e-10, io_file="critico2mc.dat",store_psi0 = true,io_ranks = "ranghi2mc.dat",io_times = "times2mc.dat")

In [15]:
dot(psi,psi0)

0.15217446904547846 - 3.2293939507024908e-9im

In [16]:
ITensor(1.)

ITensor ord=0
NDTensors.Dense{Float64, Vector{Float64}}

In [17]:
@show state("vecId",sysenv[1])

state("vecId", sysenv[1]) = ITensor ord=1
Dim 1: (dim=4|id=23|"HvS=1/2,Site")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 4-element
               0.0 + 0.0im
               0.0 + 0.0im
               0.0 + 0.0im
 1.414213562373095 + 0.0im


ITensor ord=1 (dim=4|id=23|"HvS=1/2,Site")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [18]:
V = ITensor(1.)
for j in 1:21
   print(j)
   V *= psi[j]* (j!=1 ? state("vecId",sysenv[j]) : state("vecσz",sysenv[j]))
end
scalar(V)

123456789101112131415161718192021

-0.5222493485571591 - 5.682907025898845e-9im

In [19]:
@show psi[1]

psi[1] = ITensor ord=2
Dim 1: (dim=4|id=620|"Link,u")
Dim 2: (dim=4|id=23|"HvS=1/2,Site")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 4×4
    0.5257677563818356 + 0.11022941444067924im     1.430368023670709e-7 - 7.002149527107022e-8im  -1.3547514120850946e-8 - 4.871935967233226e-8im    -0.5257677859817669 - 0.11022945051564277im
  4.405809112228313e-8 - 6.810763312418823e-8im    -0.12092841159101252 + 0.34627607777291497im       0.1395188537677348 + 0.16915329083192113im   -6.749448616099215e-8 - 4.825094412093255e-9im
 -4.019607679755466e-8 + 3.511982988302459e-8im     0.21545558409055984 - 0.0407089985199687im       -0.1286537927114479 + 0.3434806576701637im    -5.534647446641072e-9 + 3.923401048452305e-8im
  -0.15929887886794245 + 0.05858723500880515im   -4.0770210812353646e-8 - 1.57986749340376e-8im    1.3819272683773203e-8 - 5.013724040560859e-9im   -0.15929887127044726 + 0.058587222373162544im


ITensor ord=2 (dim=4|id=620|"Link,u") (dim=4|id=23|"HvS=1/2,Site")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [20]:
@show psi0[1]

psi0[1] = ITensor ord=2
Dim 1: (dim=4|id=23|"HvS=1/2,Site")
Dim 2: (dim=1|id=465|"Link,l=1")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 4×1
 0.7071067811865476 + 0.0im
                0.0 + 0.0im
                0.0 + 0.0im
 0.7071067811865475 + 0.0im


ITensor ord=2 (dim=4|id=23|"HvS=1/2,Site") (dim=1|id=465|"Link,l=1")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [21]:
V = ITensor(1.)
for j in 1:21
   V *= psi[j]* (j!=2 ? state("vecId",sysenv[j]) : state("vecId",sysenv[j]))
end
scalar(V)


0.9897591384705287 + 7.083778029450728e-9im